**CHECK FOR GPU**

In [ ]:
!nvidia-smi

**CONNECT TO GOOGLE DRIVE**

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**IMPORT THE LIBRARIES**

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

**INITIATE DEVICE**

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**1. SQUEEZENET MODEL INITIALIZATION**

In [ ]:
# NE KONVERGIRA / OVERFITTING
from torchvision.models import squeezenet1_0
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

model = squeezenet1_0(pretrained=True)
model.num_classes = 100
model.classifier[1] = nn.Conv2d(512, 100, kernel_size=(1, 1), stride=(1, 1))

model.train()

criterion = CrossEntropyLoss()
optimizer = optimizer = optim.Adam(model.parameters(), lr=1E-5)

model.to(device)
print(model)


**2. RESNET34 MODEL INITIALIZATION**

In [ ]:
# TRAINED

from torchvision.models import resnet34
from torch.nn import CrossEntropyLoss

model = resnet34(pretrained=True)

# number of classes
model.fc = torch.nn.Linear(model.fc.in_features, 100)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optimizer = optim.Adam(model.parameters(), lr=1E-4)

model.to(device)

**3. RESNET18 MODEL INITIALIZATION**

In [ ]:
# TRAINED

from torchvision.models import resnet18
from torch.nn import CrossEntropyLoss

model = resnet18(pretrained=True)

# number of classes
model.fc = torch.nn.Linear(model.fc.in_features, 100)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

model.to(device)

**4. RESNET50 MODEL INITIALIZATION**

In [ ]:
# TRAINED

from torchvision.models import resnet50
from torch.nn import CrossEntropyLoss

model = resnet50(pretrained=True)

# number of classes
model.fc = torch.nn.Linear(model.fc.in_features, 100)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optimizer = optim.Adam(model.parameters(), lr=1E-4)

model.to(device)

**5. VGG MODEL INITIALIZATION**

In [ ]:
# NE KONVERGIRA / OVERFITTING

from torchvision.models import vgg16
from torch.nn import CrossEntropyLoss

model = vgg16(pretrained=True)

# veliko št. nodeov na dense layerju mreže --> overfitting, močne izgube med trainingom --> update to 1024
#model.classifier[3] = nn.Linear(4096,4096)
model.classifier[4] = nn.Linear(4096,1024)
# number of classes 
model.classifier[6] = nn.Linear(1024,100) 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1E-5)

model.to(device)
print(model)

**LOAD DATASET**

In [7]:
#train_dir = "/mydrive/perfectly_detected_ears/train_pytorch/"
#test_dir = "/mydrive/perfectly_detected_ears/test_pytorch/"

# MY YOLO4 EXTRACTED DATASET
train_dir = "/mydrive/yolov4_ears/train_pytorch/"
test_dir = "/mydrive/yolov4_ears/test_pytorch/"

train_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.RandomVerticalFlip(0.4), 
                                       transforms.RandomHorizontalFlip(0.4), 
                                       transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
                                       transforms.ColorJitter(brightness=.3, saturation=.2, contrast=.5),
                                       transforms.ToTensor(),
                                       transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3)),
                                       transforms.Normalize([0.5442, 0.3906, 0.3292], 
                                                            [0.2593, 0.2215, 0.2166])
                                       ])

test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5506, 0.3965, 0.3385],
                                                           [0.2567, 0.2192, 0.2177])
                                      ])

test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)



# TO USE IN TRAIN AND TEST TRANSFORMS
#mean, std = get_mean_std(trainloader)
#print(mean, std)
#mean, std = get_mean_std(testloader)
#print(mean, std)


**REUPLOAD POTENTIAL HALF-TRAINED MODEL**

In [ ]:
model_save_name = 'classifier_resnet34_30.pt'
PATH = "/mydrive/" + model_save_name
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
train_losses = checkpoint['train_loss']
test_losses = checkpoint['test_loss']
model.train()

**TRAIN**

In [ ]:
epochs = 100
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []

for epoch in range(epochs):
    train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ps = torch.exp(logps)
        top_p , top_class = ps.topk(1,dim=1)
        equals = top_class == labels.view(*top_class.shape)
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {100 * accuracy/len(testloader):.3f}%")
            running_loss = 0
            model.train()
            if ((epoch%50 == 0 and epoch != 0) or (epoch == 30)):
              model_save_name = 'classifier_resnet34_'+str(epoch)+'.pt'
              PATH = "/mydrive/" + model_save_name
              state = {
                  'epoch': epoch,
                  'state_dict': model.state_dict(),
                  'optimizer': optimizer.state_dict(),
                  'train_loss': train_losses,
                  'test_loss': test_losses
              }
              torch.save(state, PATH)


**SAVE THE MODEL**

In [ ]:
model_save_name = 'classifier_resnet50_50_jitter_crop.pt'
PATH = "/mydrive/" + model_save_name
state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'train_loss': train_losses,
    'test_loss': test_losses
}
torch.save(state, PATH)

**PLOT LOSS**

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

**TEST ON VALIDATION DATASET**

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
       inputs, labels = inputs.to(device), labels.to(device)
       logps = model(inputs)
       _, predicted = torch.max(logps, 1)
       total += labels.size(0)
       correct += (predicted == labels).sum().item()
       
print(total)
print(correct)
print(f'Accuracy of the network on the test images: {100 * correct // total} %')

**CALCULATE MEAN AND STANDARD DEVIATION OF DATASET**

In [ ]:
def get_mean_std(loader):
  channel_sum, channel_squared_sum, num_batches = 0,0,0

  for data, _ in loader:
    channel_sum += torch.mean(data, dim=[0,2,3])
    channel_squared_sum += torch.mean(data**2, dim=[0,2,3])
    num_batches += 1
  
  mean = channel_sum/num_batches
  std = (channel_squared_sum/num_batches - mean**2)**0.5

  return mean, std